# Démonstration en temps réel de suivi d'objet

### Contrôle du robot pour le suivi d'un objet

Nous voulons que le robot puisse suivre un objet spécifique. Pour cela, nous allons mettre en place les étapes suivantes :

1.  Détecter les objets d'une classe spécifique
2.  Sélectionner l'objet le plus proche du centre du champ de vision de la caméra. Cela sera notre objet "cible"
3.  Piloter le robot vers la cible détectée, sinon errer
4.  Si le robot est bloqué par un obstacle, tourner à gauche

Fichier des labels : https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_label_map.pbtxt

Nous allons également mettre en place des widgets permettant de contrôler la vitesse du robot, de choisir l'objet cible, ainsi que le gain en vitesse avec lequel le robot tournera pour aller vers la cible selon la distance avec celle-ci depuis le centre de la caméra.

In [ ]:
from jetbot import Robot

robot = Robot()

La classe ci-dessous reprend la classe vue dans le carnet précédent (CameraTempsReel) mais en y ajoutant le pilotage du robot et la détection des obstacles.

In [ ]:
import tensorrt as trt
import numpy as np
import pycuda.driver as cuda
import threading
import ctypes
import time
import traitlets
import atexit
import cv2

class TRTInference(threading.Thread):
    def __init__(self,repertoire_engine_detection_objet, repertoire_labels, repertoire_engine_collision,
                 widget_image,widget_bloque,widget_label,text_label_widget,vitesse_widget,gain_widget,widget_seuil_proba,widget_seuil_proba_bloque,
                 type_camera="CSI",capture_device="0",capture_width="320",capture_height="320",
                 display_width="320",display_height="320",fps="30",flip=0,input_detect_width=320,input_detect_height=320):
        threading.Thread.__init__(self)
        self.widget_image = widget_image
        self.widget_bloque = widget_bloque
        self.type_camera = type_camera
        self.capture_device = capture_device
        self.capture_width = capture_width
        self.capture_height = capture_height
        self.display_width = display_width
        self.display_height = display_height
        self.widget_label = widget_label
        self.text_label_widget = text_label_widget
        self.vitesse_widget = vitesse_widget
        self.widget_seuil_proba = widget_seuil_proba
        self.widget_seuil_proba_bloque = widget_seuil_proba_bloque
        self.gain_widget = gain_widget
        self.fps = fps
        self.flip = flip
        self.camera_on = False
        self.input_detect_width = input_detect_width
        self.input_detect_height = input_detect_height
        self.centre_1 = [input_detect_width/2,input_detect_height/2]

        # Initialisation des variables de la caméra
        self._running = False
        self.image = np.zeros((self.display_height, self.display_width, 3), dtype=np.uint8)
        if self.type_camera.find("CSI")>=0:
            self.cap = cv2.VideoCapture(self._gstreamer_pipeline_CSI(),cv2.CAP_GSTREAMER)
        else:
            self.cap = cv2.VideoCapture(self._gstreamer_pipeline_USB(),cv2.CAP_GSTREAMER)
        if self.cap.isOpened():
            print("Caméra initialisée")
        else:
            print("Erreur d'ouverture du flux vidéo")
        atexit.register(self.cap.release)

        # Initialisation du runtime TensorRT
        self.logger = trt.Logger(trt.Logger.INFO)
        trt.init_libnvinfer_plugins(self.logger, namespace="")
        self.runtime = trt.Runtime(self.logger)
        
        ################################################################
        # Chargement du moteur de détection d'objets
        print("Chargement du moteur de détection d'objets...")
        with open(repertoire_engine_detection_objet, "rb") as f:
            self.engine_detection_objet = self.runtime.deserialize_cuda_engine(f.read())
        
        self.context_detection_objet = self.engine_detection_objet.create_execution_context()

        #Initialisation du context Cuda et du contexte TensorRT 
        cuda.init()
        self.cudactx = cuda.Device(0).retain_primary_context()
        self.cudactx.push()
        
        # Réservation de la mémoire pour l'entrée
        print("Allocation mémoire...")
        size_input = trt.volume(self.engine_detection_objet.get_binding_shape(0))*self.engine_detection_objet.max_batch_size
        self.input_host_mem_detection_objet = cuda.pagelocked_empty(size_input, np.float32)
        self.input_device_mem_detection_objet = cuda.mem_alloc(self.input_host_mem_detection_objet.nbytes)

        # Réservation de la mémoire pour les sorties
        self.output_device_mem_detection_objet = [];
        format_sorties = [];
        types_sorties = [];

        for i in range(self.engine_detection_objet.num_bindings):
            if not self.engine_detection_objet.binding_is_input(i):
                size_output = trt.volume(self.engine_detection_objet.get_binding_shape(i))*self.engine_detection_objet.max_batch_size
                output_hm = cuda.pagelocked_empty(size_output,trt.nptype(self.engine_detection_objet.get_binding_dtype(i)))
                self.output_device_mem_detection_objet.append(cuda.mem_alloc(output_hm.nbytes))
                format_sorties.append(self.engine_detection_objet.get_binding_shape(i))
                types_sorties.append(trt.nptype(self.engine_detection_objet.get_binding_dtype(i)))

        # Récupère les adresses en GPU des buffers entrées / sorties
        binding_entree = int(self.input_device_mem_detection_objet)
        binding_sorties = []

        for output_ in self.output_device_mem_detection_objet:
            binding_sorties.append(int(output_))
        self.bindings_detection_objet = [binding_entree, binding_sorties[0],binding_sorties[1],binding_sorties[2],binding_sorties[3]]

        # Allocation de la mémoire hote pour les sorties
        self.output_host_mem_detection_objet = []
        for i in range(len(self.output_device_mem_detection_objet)):
            self.output_host_mem_detection_objet.append(np.zeros(format_sorties[i],types_sorties[i]))
        
        # Input tensor
        self.image = np.zeros((self.input_detect_width,self.input_detect_height,3), dtype=trt.nptype(self.engine_detection_objet.get_binding_dtype(0)))

        # Initialisation des labels
        self.classes = self.read_label_map(repertoire_labels)

        self.cudactx.pop()
        
        #####################################################################
        # Chargement du moteur détection de collisions
        print("Chargement du moteur de détection de collisions...")
        with open(repertoire_engine_collision, "rb") as f:
            self.engine_collision = self.runtime.deserialize_cuda_engine(f.read())
        
        self.context_collision = self.engine_collision.create_execution_context()
        self.context_collision.debug_sync = True

        #Initialisation du context Cuda 
        self.cudactx = cuda.Device(0).retain_primary_context()
        self.cudactx.push()
        
        # Réservation de la mémoire pour l'entrée et la sortie
        print("Allocation mémoire...")
        size_input = trt.volume(self.engine_collision.get_binding_shape(0))*self.engine_collision.max_batch_size
        size_output = trt.volume(self.engine_collision.get_binding_shape(1))* self.engine_collision.max_batch_size
        self.input_host_mem_collision = cuda.pagelocked_empty(size_input, np.float32)
        self.input_device_mem_collision = cuda.mem_alloc(self.input_host_mem_collision.nbytes)
        self.output_host_mem_collision = cuda.pagelocked_empty(size_output, np.float32)
        self.output_device_mem_collision = cuda.mem_alloc(self.output_host_mem_collision.nbytes)
        self.bindings_collision = [int(self.input_device_mem_collision), int(self.output_device_mem_collision)]
        self.cudactx.pop()
        
    # Lectures de labels
    def read_label_map(self,label_map_path):
        item_id = None
        item_name = None
        items = {}

        with open(label_map_path, "r") as file:
            for line in file:
                line.replace(" ", "")
                if line == "item{":
                    pass
                elif line == "}":
                    pass
                elif "id" in line:
                    item_id = int(line.split(":", 1)[1].strip())
                elif "display_name" in line:
                    item_name = line.split(" ")[-1].replace("\"", " ").strip()
                if item_id is not None and item_name is not None:
                    items[item_id] = item_name
                    item_id = None
                    item_name = None
        return items
    
    # Calcul des coordonnées x,y du centre de la boite
    def calcul_centre_boite(self,detection):
        ymin = int(self.input_detect_height * detection[0])
        xmin = int(self.input_detect_width * detection[1])
        ymax = int(self.input_detect_height * detection[2])
        xmax = int(self.input_detect_width * detection[3])
        centre_x = (xmin + xmax) / 2.0
        centre_y = (ymin + ymax) / 2.0
        return [centre_x, centre_y]

    # Calcul de la norme du vecteur position entre le centre de la boite et le centre de la caméra
    def norme(self,vec):
        return np.sqrt((vec[0]-self.input_detect_width/2)**2 + (vec[1]-self.input_detect_height/2)**2)

    # Détection de la boite avec la plus haute probabilité
    def detect_le_plus_probable(self,detections,index_detections):
        det_plus_probable = None
        proba = 0
        i = 0
        index_boite = None
        for det in detections:
            if det_plus_probable is None:
                det_plus_probable = det
                index_boite = index_detections[i]
                proba = self.output_host_mem_detection_objet[2][0,index_detections[i]]
            elif self.output_host_mem_detection_objet[2][0,index_detections[i]] > proba:
                proba = self.output_host_mem_detection_objet[2][0,index_detections[i]]
                det_plus_probable = det
                index_boite = index_detections[i]
            i = i + 1
        return det_plus_probable, index_boite

    # Inférence
    def Calcul(self):
        ######################################################
        # Détection de collisions
        #######################################################
        img_224 = cv2.resize(self.image,dsize=(224,224))
        x = img_224.astype(np.float32)
        x = np.expand_dims(x,axis=0)                    # (1,224,224,3)
        np.copyto(self.input_host_mem_collision,x.ravel())

        # Transfert de l'entrée vers le GPU
        self.cudactx = cuda.Device(0).retain_primary_context()
        self.cudactx.push()
        cuda.memcpy_htod(self.input_device_mem_collision, self.input_host_mem_collision)

        # Appel du modèle
        self.context_collision.execute(batch_size=1, bindings=self.bindings_collision)
        
        # Récupération de la sortie
        cuda.memcpy_dtoh(self.output_host_mem_collision, self.output_device_mem_collision)
        self.cudactx.pop()

        proba_bloquer = float(np.asarray(self.output_host_mem_collision[0]))
        self.widget_bloque.value = proba_bloquer
        
        ################################################
        # Détection d'objets
        ################################################
        # Copie de l'image dans le tenseur d'entrée
        x = self.image.astype(np.float32)
        x = np.expand_dims(x,axis=0)                    # (1,input_detect_width,input_detect_height,3)
        np.copyto(self.input_host_mem_detection_objet,x.ravel())
        
        # Transfert de l'entrée vers le GPU
        self.cudactx = cuda.Device(0).retain_primary_context()
        self.cudactx.push()
        cuda.memcpy_htod(self.input_device_mem_detection_objet, self.input_host_mem_detection_objet)
        
        # Appel du modèle
        self.context_detection_objet.execute(batch_size=1, bindings=self.bindings_detection_objet)
        
        # Récupération des sorties
        for i in range(len(self.output_host_mem_detection_objet)):
            cuda.memcpy_dtoh(self.output_host_mem_detection_objet[i], self.output_device_mem_detection_objet[i])
        self.cudactx.pop()
        
        ##############################################
        # Traitement des résultats
        ##############################################
       
        # Affiche le nom de l'objet en cours de traitement
        self.text_label_widget.value = str(self.classes.get(int(self.widget_label.value)))

        # Récupère le ou les boites des objets de la classe à détecter
        index = 0
        detect = []
        detect_idx = []
        for boite in self.output_host_mem_detection_objet[1][0,:,:]:
            classe = self.output_host_mem_detection_objet[3][0,index]+1
            if classe == int(self.widget_label.value) and self.output_host_mem_detection_objet[2][0,index] > self.widget_seuil_proba.value:
                detect.append(boite)
                detect_idx.append(index)
            index = index + 1
                
        # Récupère la boite la plus probable
        boite_la_plus_probable, index = self.detect_le_plus_probable(detect,detect_idx)

        # Si un objet à suivre a été détecté
        # Affiche un rectangle en rouge sur l'objet à suivre
        # et avance vers la cible 
        if boite_la_plus_probable is not None and proba_bloquer < self.widget_seuil_proba_bloque.value:
            # Tracé du rectangle
            ymin = int(self.input_detect_height * boite_la_plus_probable[0])
            xmin = int(self.input_detect_width * boite_la_plus_probable[1])
            ymax = int(self.input_detect_height * boite_la_plus_probable[2])
            xmax = int(self.input_detect_width * boite_la_plus_probable[3])
            cv2.rectangle(self.image, (xmin,ymin),
                              (xmax, ymax),
                              (0, 0, 255), 1)
            cv2.putText(self.image,
                                str(self.classes.get(1+self.output_host_mem_detection_objet[3][0,index])),
                                (xmin,ymin+20),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                0.6,
                                (0, 0, 255),
                                1,
                                cv2.LINE_AA)
            cv2.putText(self.image,
                                str(self.output_host_mem_detection_objet[2][0,index]),
                                (xmin,ymin+40),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                0.4,
                                (0, 0, 255),
                                1,
                                cv2.LINE_AA)
            
            # Avance le robot vers la cible avec vitesse propotionnelle à la distance de celle-ci
            centre = self.calcul_centre_boite(boite_la_plus_probable)
            if self.vitesse_widget.value == 0.0:
                robot.set_motors(0.0,0.0)
            else:
                ecart = centre[0]-self.input_detect_width/2
                robot.set_motors(
                    float(self.vitesse_widget.value + self.gain_widget.value * ecart/self.input_detect_width),
                    float(self.vitesse_widget.value - self.gain_widget.value * ecart/self.input_detect_width)
                    )
        
        # Si aucun objet à suivre n'a été détecté et qu'il n'y a aucun obstacle
        elif proba_bloquer < self.widget_seuil_proba_bloque.value:
            # Affiche les rectangles en bleu sur les 3 premiers objets détectés
            for i in range(3):
                ymin = int(self.input_detect_height * self.output_host_mem_detection_objet[1][0,i,0])
                xmin = int(self.input_detect_width * self.output_host_mem_detection_objet[1][0,i,1])
                ymax = int(self.input_detect_height * self.output_host_mem_detection_objet[1][0,i,2])
                xmax = int(self.input_detect_width * self.output_host_mem_detection_objet[1][0,i,3])

                cv2.rectangle(self.image, (xmin,ymin),
                                  (xmax, ymax),
                                  (255, 0, 0), 1)
                cv2.putText(self.image,
                                str(self.classes.get(1+self.output_host_mem_detection_objet[3][0,i])),
                                (xmin,ymin+20),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                0.6,
                                (0, 255, 0),
                                1,
                                cv2.LINE_AA)
            robot.forward(float(self.vitesse_widget.value))
            
        # Si il y a un obstacle et qu'un objet à suivre a été détecté
        elif boite_la_plus_probable is not None and proba_bloquer > self.widget_seuil_proba_bloque.value:
            centre = self.calcul_centre_boite(boite_la_plus_probable)
            # Tracé du rectangle
            ymin = int(self.input_detect_height * boite_la_plus_probable[0])
            xmin = int(self.input_detect_width * boite_la_plus_probable[1])
            ymax = int(self.input_detect_height * boite_la_plus_probable[2])
            xmax = int(self.input_detect_width * boite_la_plus_probable[3])
            cv2.rectangle(self.image, (xmin,ymin),
                              (xmax, ymax),
                              (0, 0, 255), 1)
            cv2.putText(self.image,
                                str(self.classes.get(1+self.output_host_mem_detection_objet[3][0,index])),
                                (xmin,ymin+20),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                0.6,
                                (0, 0, 255),
                                1,
                                cv2.LINE_AA)
            cv2.putText(self.image,
                                str(self.output_host_mem_detection_objet[2][0,index]),
                                (xmin,ymin+40),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                0.4,
                                (0, 0, 255),
                                1,
                                cv2.LINE_AA)
            robot.set_motors(0.0,0.0)
        
        # Obstacle et aucun objet détecté
        else:
            robot.left(float(self.vitesse_widget.value))
            
    # Lecture d'une frame
    def capture_image(self):
        re, image = self.cap.read()
        if re:
            image_resized = cv2.resize(image,(int(self.display_width),int(self.display_height)))
            return image_resized
        else:
            return self.image
        
    def run(self):
        while True:
            if self.camera_on is True:
                self.image = self.capture_image()
                self.Calcul()
                self.widget_image.value = bgr8_to_jpeg(self.image)
                time.sleep(0.001)

    # Définition du pipeline pour la caméra CSI
    def _gstreamer_pipeline_CSI(self):
        return("nvarguscamerasrc sensor-id=%d ! "
                "video/x-raw(memory:NVMM),"
                "width=(int)%d,height=(int)%d,"
                "format=(string)NV12, framerate=(fraction)%d/1 ! "
                "nvvidconv flip-method=%d ! "
                "video/x-raw,"
                "width=(int)%d,height=(int)%d,"
                "format=(string)BGRx ! videoconvert ! "
                "video/x-raw, format=(string)BGR ! "
                "appsink drop=true"
        %(self.capture_device,self.capture_width,self.capture_height,self.fps,self.flip, self.display_width,self.display_height))

    # Définition du pipeline pour la USB
    def _gstreamer_pipeline_USB(self):
        return("v4l2src device=/dev/video%d ! "
               "video/x-raw, width=(int)%d, height=(int)%d, framerate=(fraction)%d/1 ! "
               "videoflip method=%d ! "
               "videoconvert ! "
               "video/x-raw, format=(string)BGR ! appsink drop=true"
        %(self.capture_device,self.capture_width,self.capture_height,self.fps,self.flip))            

    # Routine pour arrêter le Thread
    def raise_exception(self):
        for id, thread in threading._active.items():
            if thread is self:
                thread_id = id
        res = ctypes.pythonapi.PyThreadState_SetAsyncExc(thread_id,ctypes.py_object(SystemExit))
        if res > 1:
            ctypes.pythonapi.PyThreadState_SetAsyncExc(thread_id, 0)
            print('Exception raise failure')

    def destroy(self):
        self.cudactx.detach()

Construisons maintenant l'interface de pilotage :

In [ ]:
import ipywidgets.widgets as widgets
import traitlets
from IPython.display import display
from jetbot import bgr8_to_jpeg

widget_bloque = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='bloquer')
widget_seuil_proba = widgets.FloatSlider(min=0.0, max=1.0, value=0.5, description='seuil detection')
widget_seuil_proba_bloque = widgets.FloatSlider(min=0.0, max=1.0, value=0.5, description='seuil bloquer')
image_widget = widgets.Image(format='jpeg', width=320, height=320)
label_widget = widgets.IntText(value=1, description='label cible')
text_label_widget = widgets.Text(value="")
vitesse_widget = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, description='vitesse')
gain_widget = widgets.FloatSlider(value=0.2, min=0.0, max=2.0, description='gain')


display(widgets.VBox([
    widgets.HBox([image_widget, widgets.VBox([widget_bloque,widget_seuil_proba,widget_seuil_proba_bloque])]),
    widgets.HBox([label_widget,text_label_widget]),
    vitesse_widget,
    gain_widget
]))

In [ ]:
trt_inference_wrapper = TRTInference(repertoire_engine_detection_objet="tfmodel_ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/model_ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8_nms04.engine",
                        repertoire_labels="models/research/object_detection/data/mscoco_complete_label_map.pbtxt",
                        repertoire_engine_collision="model_collision.engine",             
                        widget_image=image_widget,
                        widget_label = label_widget,
                        text_label_widget = text_label_widget,
                        widget_bloque=widget_bloque,
                        vitesse_widget=vitesse_widget,
                        widget_seuil_proba=widget_seuil_proba,
                        widget_seuil_proba_bloque = widget_seuil_proba_bloque,
                        gain_widget=gain_widget,
                        type_camera="CSI",capture_device=0,
                        capture_width=1024,capture_height=768,
                        display_width=640,display_height=640,
                        fps=20,flip=0,input_detect_width=640,input_detect_height=640)

In [ ]:
trt_inference_wrapper.start()
trt_inference_wrapper.camera_on = True

In [ ]:
trt_inference_wrapper.camera_on = False

In [ ]:
import time

time.sleep(1.0)
robot.stop()